In [1]:
# An entry from the LLM's response to a prompt will be in a format like
# [
#     {
#         "name_donor": "[NAME_DONOR]",
#         "name_recipient": "[NAME_RECIPIENT]",
#         "prior_reputation_recipient": "[PRIOR_REPUTATION_RECIPIENT]",
#         "action": "[ACTION]",
#         "topic": "[TOPIC]",
#         "donor_gender": "[DONOR_GENDER]",
#         "donor_region": "[DONOR_REGION]",
#         "recipient_gender": "[RECIPIENT_GENDER]",
#         "recipient_region": "[RECIPIENT_REGION]",
#         "response": "\"My opinion: good\"",
#         "tag": "no-topic",
#         "model_name": "gpt-3.5-turbo"
#     },
#      ...
# }

# We will save in a jsonfile all these 
# norm

# norm_donor_male_recep_male
# norm_donor_male_recep_female
# norm_donor_female_recep_male
# norm_donor_female_recep_female

# norm_donor_west_recep_mena
#..

# norm_tag_notopic
# norm_tag_notopic
# norm_tag_neutral
# norm_tag_nonneutral
# norm_tag_explicit

In [ ]:
import json
import statistics
import numpy as np

def response_to_reputation(response, prior_reputation):
    """
    Process the `response` field and convert it into a numeric value.
    For simplicity, we assume the response format is: "My opinion: good" or "My opinion: bad".
    You can adjust this logic to handle different response formats.
    """
    
    if isinstance(response, dict):
        response = response.get("content")
        
    # consider only the first line of the answer that actually contains text
    #response = response.splitlines()[0] if response else ""
    response = next((line for line in response.splitlines() if line.strip()), "")

    if "good" in response.lower() and "bad" in response.lower():
        return None # can't accept results that contain both reputations.
    elif "good" in response.lower():
        return 1
    elif "bad" in response.lower():
        return 0
    elif "neutral" in response.lower():
        return 0.5
    else:
        return None  # Invalid or unprocessable response

def calculate_norm(json_file_path, norm_name):
    """
    Calculate [x1, x2, x3, x4] (snGC, snGD, snBC, snBD) based on the conditions provided.
    
    Args:
        json_file_path (str): Path to the JSON file.
        
    Returns:
        list: [x1, x2, x3, x4]
    """
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    if(len(data) == 0):     # Case for when no data exists
        return [None, None, None, None], 0

    # Initialize sums and counts for each variable
    x1_sum, x1_count = 0, 0
    x2_sum, x2_count = 0, 0
    x3_sum, x3_count = 0, 0
    x4_sum, x4_count = 0, 0
    num_invalid_responses = 0

    # Iterate through the data entries
    for entry in data:
        prior_reputation = entry["prior_reputation_recipient"]

        response_value = response_to_reputation(entry["response"], prior_reputation)

        if response_value is None:
            num_invalid_responses += 1
            #print(entry["response"])
            continue  # Skip invalid responses

        action = entry["action"]

        # Calculate x1
        if action == "help" and prior_reputation == "good":
            x1_sum += response_value
            x1_count += 1
        # Calculate x2
        elif action == "not help" and prior_reputation == "good":
            x2_sum += response_value
            x2_count += 1
        # Calculate x3
        elif action == "help" and prior_reputation == "bad":
            x3_sum += response_value
            x3_count += 1
        # Calculate x4
        elif action == "not help" and prior_reputation == "bad":
            x4_sum += response_value
            x4_count += 1

    # Compute averages for x1, x2, x3, x4
    x1 = x1_sum / x1_count if x1_count > 0 else 0.5
    x2 = x2_sum / x2_count if x2_count > 0 else 0.5
    x3 = x3_sum / x3_count if x3_count > 0 else 0.5
    x4 = x4_sum / x4_count if x4_count > 0 else 0.5

    norm = [x1, x2, x3, x4]
    print(norm_name+": " + str(norm) + ". Invalid answers: " + str(num_invalid_responses*100/len(data)) + "%")
    return norm, num_invalid_responses, num_invalid_responses/len(data)

In [16]:
# For each model, we will save in a json file all possible norms.
# We first cycle through all the json files in the LLM response folder, and then create a single json file that has an entry for each model, with all the norms

import os
from prompt_templates import search_propriety

def find_json_files(directory):
    """
    Recursively searches the given directory for all JSON files.

    Args:
        directory (str): The path to the directory to search.

    Returns:
        list: A list of paths to JSON files found in the directory.
    """
    json_files = []

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                json_files.append(os.path.join(root, file))

    return json_files

def calculate_norm_std(model_results, norm_counts):
    """
    Calculate the weighted covariance matrix and standard deviations for multivariate Gaussian distribution.
    
    Args:
        model_results (dict): Dictionary containing the model's norms.
        norm_counts (dict): Dictionary containing the count of occurrences for each norm.
    
    Returns:
        dict: A dictionary containing the mean vector, standard deviation vector, and covariance matrix.
    """
    norm_vectors = []
    weights = []
    
    for key, value in model_results.items():
        if key.startswith("norm_") and isinstance(value, list) and all(isinstance(v, (int, float)) for v in value if v is not None):
            count = norm_counts.get(key, 0)
            if count > 0:
                norm_vectors.append([v for v in value if v is not None])
                weights.append(count)

    print(norm_vectors)
    
    if len(norm_vectors) > 1:
        #norm_matrix = np.array(norm_vectors).T  # Convert list of lists into a NumPy matrix
        #mean_vector = np.average(norm_matrix, axis=1, weights=weights)
        #cov_matrix = np.cov(norm_matrix, aweights=weights, bias=True)
        #std_vector = np.sqrt(np.diag(cov_matrix))

        norm_matrix = np.array(norm_vectors)  # Rows = samples, Columns = features
        mean_vector = np.average(norm_matrix, axis=0, weights=weights)
        cov_matrix = np.cov(norm_matrix, rowvar=False, aweights=weights, bias=False)
        std_vector = np.sqrt(np.diag(cov_matrix))
        
        return {
            "mean_vector": mean_vector.tolist(),
            "std_vector": std_vector.tolist(),
            "cov_matrix": cov_matrix.tolist()
        }
    else:
        return None  # Not enough data to compute multivariate statistics

def calculate_all_norms(results_directory, output_file_norm, output_file_error):
    all_jsons = find_json_files(results_directory)
    all_norms = []
    all_errors = []

    temp_json_file_path = "temp.json"

    for result_json in all_jsons:
        print("\nCurrent model: " + result_json)

        with open(result_json, 'r') as file:
            result = json.load(file)
        model_results = {}
        norm_counts = {}
        model_errors = {}

        model_results["model_name"] = result[1]["model_name"]
        model_errors = {"model_name": result[1]["model_name"]}
        model_results["norm"], _, model_errors["norm"] = calculate_norm(result_json, "norm")
        

        for gender_donor in ["M", "F"]:
            for gender_recipient in ["M", "F"]:
                norm_name = "norm_"+gender_donor+"_to_"+gender_recipient
                search_propriety(result_json, {"donor_gender": gender_donor, "recipient_gender": gender_recipient}, temp_json_file_path)
                model_results[norm_name], invalid_res, model_errors[norm_name] = calculate_norm(temp_json_file_path, norm_name)
                norm_counts[norm_name] = sum(1 for _ in open(temp_json_file_path)) - invalid_res

        for region_donor in ["WEST", "EASTASIA", "SUBSAHARA", "MENA"]:
            for region_recipient in ["WEST", "EASTASIA", "SUBSAHARA", "MENA"]:
                norm_name = "norm_"+region_donor+"_to_"+region_recipient
                search_propriety(result_json, {"donor_region": region_donor, "recipient_region": region_recipient}, temp_json_file_path)
                model_results[norm_name], invalid_res, model_errors[norm_name] = calculate_norm(temp_json_file_path, norm_name)
                norm_counts[norm_name] = sum(1 for _ in open(temp_json_file_path)) - invalid_res

        for tag in ["no-topic", "neutral", "non-neutral", "explicit-neutral", "explicit-non-neutral"]:
            norm_name = "norm_tag_"+tag
            search_propriety(result_json, {"tag": tag}, temp_json_file_path)
            model_results[norm_name], invalid_res, model_errors[norm_name] = calculate_norm(temp_json_file_path, norm_name)
            norm_counts[norm_name] = sum(1 for _ in open(temp_json_file_path)) - invalid_res
        
        os.remove(temp_json_file_path)
        
        # Compute and store weighted standard deviation of norms
        model_results["standard_dev"] = calculate_norm_std(model_results, norm_counts)

        all_norms.append(model_results)
        all_errors.append(model_errors)

    with open(output_file_norm, 'w') as f:
        json.dump(all_norms, f, indent=4)
    with open(output_file_error, 'w') as f:
        json.dump(all_errors, f, indent=4)
    return all_norms

In [17]:
all_norms = calculate_all_norms("../results_llm/", "all_llm_norms.json", "all_llm_errors.json")


Current model: ../results_llm/responses_gemini-1.5-pro.json
norm: [1.0, 0.02546296296296296, 0.620462962962963, 0.6994444444444444]. Invalid answers: 0.0%
norm_M_to_M: [1.0, 0.007936507936507936, 0.625, 0.6912698412698413]. Invalid answers: 0.0%
norm_M_to_F: [1.0, 0.00034722222222222224, 0.5385416666666667, 0.675]. Invalid answers: 0.0%
norm_F_to_M: [1.0, 0.05798611111111111, 0.6711805555555556, 0.7201388888888889]. Invalid answers: 0.0%
norm_F_to_F: [1.0, 0.034523809523809526, 0.6515873015873016, 0.7119047619047619]. Invalid answers: 0.0%
norm_WEST_to_WEST: [1.0, 0.022222222222222223, 0.6018518518518519, 0.7425925925925926]. Invalid answers: 0.0%
norm_WEST_to_EASTASIA: [1.0, 0.015277777777777777, 0.5625, 0.7138888888888889]. Invalid answers: 0.0%
norm_WEST_to_SUBSAHARA: [1.0, 0.0125, 0.5791666666666667, 0.6194444444444445]. Invalid answers: 0.0%
norm_WEST_to_MENA: [1.0, 0.018055555555555554, 0.6472222222222223, 0.6652777777777777]. Invalid answers: 0.0%
norm_EASTASIA_to_WEST: [1.0, 0

In [ ]:
# Cleans up the new lines so the file is more human-readable
import re

all_norms_json = json.dumps(all_norms, indent=4)

def repl_func(match):
    return " ".join(match.group().split())

# Apply regex to modify lists to be on the same line
output2 = re.sub(r"(?<=\[)[^\[\]]+(?=\])", repl_func, all_norms_json)

with open("all_llm_norms.json", "w") as f:
    f.write(output2)